In [1]:
!pip install imbalanced-learn==0.7.0 -q

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
import json
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from IPython.display import FileLink, FileLinks
from sklearn.model_selection import train_test_split

import sagemaker
from sagemaker import clarify
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.debugger import Rule, rule_configs
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

In [4]:
# Set region, boto3 and SageMaker SDK variables¶

#You can change this to a region of your choice
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = "arn:aws:iam::114169298763:role/service-role/AmazonSageMaker-ExecutionRole-20200811T163008"
account_id = boto3.client('sts').get_caller_identity()["Account"]

random_state = 42

Using AWS Region: us-west-2


In [5]:
# load stored variables
%store -r
%store

Stored variables and their in-db values:
clarify_bias_job_1_name             -> 'Clarify-Bias-2021-04-23-17-55-18-464'
data_prefix                         -> 'sagemaker-tutorial/data'
default_bucket                      -> 'sagemaker-us-west-2-114169298763'
header                              -> ['LABEL', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIA
hyperparameters                     -> {'max_depth': '5', 'eta': '0.2', 'gamma': '4', 'mi
local_data_dir                      -> '../data'
local_processed_path                -> '../data/df_processed.csv'
local_raw_path                      -> '../data/dataset.csv'
mpg_name                            -> 'sagemaker-tutorial'
prefix                              -> 'sagemaker-tutorial'
s3_raw_data                         -> 's3://sagemaker-us-west-2-114169298763/sagemaker-t
test_data_uri                       -> 's3://sagemaker-us-west-2-114169298763/sagemaker-t
train_data_uri                      -> 's3://sagemaker-us-west-2-114169298763/sagemake

In [6]:
df = pd.read_csv(local_processed_path)
df.head()

,LABEL,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,1,20000,2,2,1,24,2,2,-1,-1,...,689,0,0,0,0,689,0,0,0,0
1,1,120000,2,2,2,26,-1,2,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,0,90000,2,2,2,34,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,0,50000,2,2,1,37,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,0,50000,1,2,1,57,-1,0,-1,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679


In [7]:
sm = SMOTE(random_state=random_state)

df_resampled, _ = sm.fit_resample(df, df['SEX'])

In [8]:
# Check the gender balance
df_resampled['SEX'].value_counts()

2    18112
1    18112
Name: SEX, dtype: int64

In [9]:
X_train, X_test = train_test_split(df_resampled, test_size=0.2, random_state=random_state)
X_train, X_val = train_test_split(df_resampled, test_size=0.2, random_state=random_state)

In [10]:
X_train.to_csv(f'{local_data_dir}/train_res.csv', header=False, index=False)

response = sagemaker_session.upload_data(f'{local_data_dir}/train_res.csv',
                                         bucket=default_bucket,
                                         key_prefix=data_prefix)
train_res_data_uri = response
%store train_res_data_uri

Stored 'train_res_data_uri' (str)


In [11]:
# training data with header - needed for model monitor baseline job
X_train.to_csv(f'{local_data_dir}/train_res_header.csv', header=True, index=False)

response = sagemaker_session.upload_data(f'{local_data_dir}/train_res_header.csv',
                                         bucket=default_bucket,
                                         key_prefix=data_prefix)
train_res_data_header_uri = response
%store train_res_data_header_uri

Stored 'train_res_data_header_uri' (str)


In [12]:
X_val.to_csv(f'{local_data_dir}/validation_res.csv', header=False, index=False)

response = sagemaker_session.upload_data(f'{local_data_dir}/validation_res.csv',
                                         bucket=default_bucket,
                                         key_prefix=data_prefix)
validation_res_data_uri = response
%store validation_res_data_uri

Stored 'validation_res_data_uri' (str)


In [13]:
X_test.to_csv(f'{local_data_dir}/test_res.csv', header=False, index=False)

response = sagemaker_session.upload_data(f'{local_data_dir}/test_res.csv',
                                         bucket=default_bucket,
                                         key_prefix=data_prefix)
test_res_data_uri = response
%store test_res_data_uri

Stored 'test_res_data_uri' (str)


In [15]:
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
content_type = "text/csv"
estimator_output_path = f's3://{default_bucket}/{prefix}/training_jobs'

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")


# construct a SageMaker estimator that calls the xgboost-container
xgb_estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container,
                                          hyperparameters=hyperparameters,
                                          role="arn:aws:iam::114169298763:role/service-role/AmazonSageMaker-ExecutionRole-20200811T163008",
                                          instance_count=train_instance_count,
                                          instance_type=train_instance_type,
                                          volume_size=5,  # 5 GB
                                          output_path=estimator_output_path,
                                         )

In [16]:
hyperparameter_ranges = {'max_depth': IntegerParameter(1, 10),
                     'eta': ContinuousParameter(0, 1),
                     'gamma': ContinuousParameter(0, 5),
                    'alpha': ContinuousParameter(0, 2)
                    }

In [18]:
objective_metric_name = 'validation:f1'

In [19]:
tuner = HyperparameterTuner(xgb_estimator,
                        objective_metric_name,
                        hyperparameter_ranges,
                        max_jobs=4,
                        max_parallel_jobs=2)

# You can increase the number of jobs, etc. I set them to 10, 4 for the demo purpose

In [20]:
# define the data type and paths to the training and validation datasets
train_input = TrainingInput(train_data_uri, content_type="text/csv")
validation_input = TrainingInput(validation_data_uri, content_type="text/csv")

# execute the XGBoost training job
tuner.fit({'train': train_input,
                   'validation': validation_input
                  }
                   )

....................................................................................................................................................!


In [21]:
training_smote_job_name = tuner.best_training_job()
%store training_smote_job_name

Stored 'training_smote_job_name' (str)


In [22]:
training_job_info = sagemaker_boto_client.describe_training_job(TrainingJobName=training_smote_job_name)

In [23]:
# store hyperparameters for best training job
best_job_hp = training_job_info['HyperParameters']
%store best_job_hp

Stored 'best_job_hp' (dict)


In [24]:
model_2_name = f'{prefix}-xgboost-smote'


model_matches = sagemaker_boto_client.list_models(NameContains=model_2_name)['Models']

if not model_matches:

    model_2 = sagemaker_session.create_model_from_job(
        name=model_2_name,
        training_job_name=training_job_info['TrainingJobName'],
        role=sagemaker_role,
        image_uri=training_job_info['AlgorithmSpecification']['TrainingImage'])
    %store model_2_name

else:

    print(f"Model {model_2_name} already exists.")

Stored 'model_2_name' (str)


In [25]:
%store model_2_name

Stored 'model_2_name' (str)


In [26]:
training_data_s3_uri = training_job_info['InputDataConfig'][0]['DataSource']['S3DataSource']['S3Uri']

matching_artifacts = list(sagemaker.lineage.artifact.Artifact.list(
    source_uri=training_data_s3_uri,
    sagemaker_session=sagemaker_session))

if matching_artifacts:
    training_data_artifact = matching_artifacts[0]
    print(f'Using existing artifact: {training_data_artifact.artifact_arn}')
else:
    training_data_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name='TrainingData',
        source_uri=training_data_s3_uri,
        artifact_type='Dataset',
        sagemaker_session=sagemaker_session)
    print(f'Create artifact {training_data_artifact.artifact_arn}: SUCCESSFUL')

Using existing artifact: arn:aws:sagemaker:us-west-2:114169298763:artifact/415f9f6f7a37326cc2e4d866bac4dfc2


In [27]:
trained_model_s3_uri = training_job_info['ModelArtifacts']['S3ModelArtifacts']

matching_artifacts = list(sagemaker.lineage.artifact.Artifact.list(
    source_uri=trained_model_s3_uri,
    sagemaker_session=sagemaker_session))

if matching_artifacts:
    model_artifact = matching_artifacts[0]
    print(f'Using existing artifact: {model_artifact.artifact_arn}')
else:
    model_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name='TrainedModel',
        source_uri=trained_model_s3_uri,
        artifact_type='Model',
        sagemaker_session=sagemaker_session)
    print(f'Create artifact {model_artifact.artifact_arn}: SUCCESSFUL')

Using existing artifact: arn:aws:sagemaker:us-west-2:114169298763:artifact/84d2636c4763533f2a2540f852171e30


In [28]:
trial_component = sagemaker_boto_client.describe_trial_component(TrialComponentName=tuner.best_training_job()+'-aws-training-job')
trial_component_arn = trial_component['TrialComponentArn']

In [29]:
# Input artifacts
input_artifacts = [training_data_artifact]

for a in input_artifacts:
    try:
        sagemaker.lineage.association.Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type='ContributedTo',
            sagemaker_session=sagemaker_session)
        print(f"Associate {trial_component_arn} and {a.artifact_arn}: SUCCEESFUL\n")
    except:
        print(f"Association already exists between {trial_component_arn} and {a.artifact_arn}.\n")

Association already exists between arn:aws:sagemaker:us-west-2:114169298763:experiment-trial-component/sagemaker-xgboost-210423-1840-001-8f062f67-aws-training-job and arn:aws:sagemaker:us-west-2:114169298763:artifact/415f9f6f7a37326cc2e4d866bac4dfc2.



In [30]:
# Output artifacts

output_artifacts = [model_artifact]

for artifact_arn in output_artifacts:
    try:
        sagemaker.lineage.association.Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type='Produced',
            sagemaker_session=sagemaker_session)
        print(f"Associate {trial_component_arn} and {a.artifact_arn}: SUCCEESFUL\n")
    except:
        print(f"Association already exists between {trial_component_arn} and {a.artifact_arn}.\n")

Association already exists between arn:aws:sagemaker:us-west-2:114169298763:experiment-trial-component/sagemaker-xgboost-210423-1840-001-8f062f67-aws-training-job and arn:aws:sagemaker:us-west-2:114169298763:artifact/415f9f6f7a37326cc2e4d866bac4dfc2.



In [31]:
model_metrics_report = {'classification_metrics': {}}
for metric in training_job_info['FinalMetricDataList']:
    stat = {metric['MetricName']: {'value': metric['Value']}}
    model_metrics_report['classification_metrics'].update(stat)

with open('training_metrics.json', 'w') as f:
    json.dump(model_metrics_report, f)

metrics_s3_key = f"{prefix}/training_jobs/{training_job_info['TrainingJobName']}/training_metrics.json"
s3_client.upload_file(Filename='training_metrics.json', Bucket=default_bucket, Key=metrics_s3_key)

In [32]:
model_metrics = {
    'ModelQuality': {
        'Statistics': {
            'ContentType': 'application/json',
            'S3Uri': f's3://{default_bucket}/{prefix}/{metrics_s3_key}'
        }
    }
}

In [33]:
inference_spec ={    
    "InferenceSpecification": {
        "Containers" : [{
            "Image": training_job_info['AlgorithmSpecification']['TrainingImage'],
            "ModelDataUrl": training_job_info['ModelArtifacts']['S3ModelArtifacts']
        }],
        "SupportedTransformInstanceTypes": ["ml.m4.xlarge"],
        "SupportedRealtimeInferenceInstanceTypes": ["ml.m4.xlarge"],
        "SupportedContentTypes": ['text/csv'],
        "SupportedResponseMIMETypes": ['text/csv']
    }
}



# {'ModelDataUrl': }

In [34]:
mp_input_dict = {
    'ModelPackageGroupName': mpg_name,
    'ModelPackageDescription': 'XGBoost classifier with SMOTE',
    'ModelApprovalStatus': 'PendingManualApproval',
    'ModelMetrics': model_metrics
}

mp_input_dict.update(inference_spec)
mp2_response = sagemaker_boto_client.create_model_package(**mp_input_dict)
mp2_arn = mp2_response['ModelPackageArn']
%store mp2_arn

Stored 'mp2_arn' (str)


In [35]:
# Check status of model package creation¶

mp_info = sagemaker_boto_client.describe_model_package(ModelPackageName=mp2_response['ModelPackageArn'])
mp_status = mp_info['ModelPackageStatus']

while mp_status not in ['Completed', 'Failed']:
    time.sleep(5)
    mp_info = sagemaker_boto_client.describe_model_package(ModelPackageName=mp2_response['ModelPackageArn'])
    mp_status = mp_info['ModelPackageStatus']
    print(f'model package status: {mp_status}')
print(f'model package status: {mp_status}')

model package status: Completed


In [36]:
sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList']

[{'ModelPackageGroupName': 'sagemaker-tutorial',
  'ModelPackageVersion': 2,
  'ModelPackageArn': 'arn:aws:sagemaker:us-west-2:114169298763:model-package/sagemaker-tutorial/2',
  'ModelPackageDescription': 'XGBoost classifier with SMOTE',
  'CreationTime': datetime.datetime(2021, 4, 23, 19, 0, 54, 649000, tzinfo=tzlocal()),
  'ModelPackageStatus': 'Completed',
  'ModelApprovalStatus': 'PendingManualApproval'},
 {'ModelPackageGroupName': 'sagemaker-tutorial',
  'ModelPackageVersion': 1,
  'ModelPackageArn': 'arn:aws:sagemaker:us-west-2:114169298763:model-package/sagemaker-tutorial/1',
  'ModelPackageDescription': 'XGBoost classifier to detect insurance fraud.',
  'CreationTime': datetime.datetime(2021, 4, 23, 18, 20, 44, 138000, tzinfo=tzlocal()),
  'ModelPackageStatus': 'Completed',
  'ModelApprovalStatus': 'PendingManualApproval'}]

In [37]:
clarify_processor = clarify.SageMakerClarifyProcessor(role=sagemaker_role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

bias_report_output_path = f's3://{default_bucket}/{prefix}/clarify-output/bias_2'
bias_data_config = clarify.DataConfig(s3_data_input_path=train_data_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='LABEL',
                                      headers=header,
                                      dataset_type='text/csv')

model_config = clarify.ModelConfig(model_name=model_2_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='SEX',
                                facet_values_or_threshold=[1],
                                group_name='AGE'
                                )


clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

clarify_bias_job_2_name = clarify_processor.latest_job.name
# clarify_bais_job_2_name = 'Clarify-Bias-2021-04-19-18-43-09-582'
%store clarify_bias_job_2_name


Job Name:  Clarify-Bias-2021-04-23-19-01-35-414
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-114169298763/sagemaker-tutorial/data/train.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-114169298763/sagemaker-tutorial/clarify-output/bias_2/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-114169298763/sagemaker-tutorial/clarify-output/bias_2', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
...........................INFO:sag

In [41]:
s3_client.download_file(Bucket=default_bucket,
                        Key=f'{prefix}/clarify-output/bias_2/report.pdf',
                        Filename='../sagemaker-workshop/outputs/clarify_output/bias_2_report.pdf')
print(f'Downloaded clarify report from previous Clarify job: {clarify_bias_job_2_name}')
display("Click link below to view the Clarify repot.", FileLink("../sagemaker-workshop/outputs/clarify_output/bias_2_report.pdf"))

FileNotFoundError: [Errno 2] No such file or directory: '../sagemaker-workshop/outputs/clarify_output/bias_2_report.pdf.ed1B10B0'

In [42]:
shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[X_train.median().values[1:].tolist()],
    num_samples=100,
    agg_method='mean_abs')

explainability_output_path = f's3://{default_bucket}/{prefix}/clarify-output/explainability'
explainability_data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_res_data_uri,
    s3_output_path=explainability_output_path,
    label='LABEL',
    headers=header,
    dataset_type='text/csv')

# # un-comment the code below to run the whole job
# clarify_processor.run_explainability(
#     data_config=explainability_data_config,
#     model_config=model_config,
#     explainability_config=shap_config)

# clarify_expl_job_name = clarify_processor.latest_job.name
# %store clarify_expl_job_name
# print(f'Clarify job {clarify_expl_job_name} ran successfully.')

In [43]:
s3://{default_bucket}/{prefix}/clarify-output/explainability

SyntaxError: invalid syntax (<ipython-input-43-45ac420ff4ef>, line 1)

In [44]:
if 'clarify_expl_job_name' in locals():
    s3_client.download_file(
        Bucket   = default_bucket,
        Key      = f'{prefix}/clarify-output/explainability/analysis.json',
        Filename = '../outputs/clarify_output/explainability_analysis.json'
    )
    print(f'Downloaded analysis from previous Clarify job: {clarify_expl_job_name}\n')
else:
    print(f'Loading pre-generated analysis file...\n')

with open('../outputs/clarify_output/explainability_analysis.json', 'r') as f:
        analysis_result = json.load(f)

shap_values = pd.DataFrame(analysis_result['explanations']['kernel_shap']["label0"])
importances = shap_values['global_shap_values'].sort_values(ascending=False)


fig, ax = plt.subplots()
n = 10
y_pos = np.arange(n)
importance_scores = importances.values[:n]
y_label = importances.index[:n]
ax.barh(y_pos, importance_scores, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(y_label)
ax.invert_yaxis()  
ax.set_xlabel('SHAP Value (impact on model output)');

Loading pre-generated analysis file...



FileNotFoundError: [Errno 2] No such file or directory: '../outputs/clarify_output/explainability_analysis.json'